In [1]:
import requests

def get_search_result_count(query: str) -> int:
    url = "https://www.ebi.ac.uk/europepmc/webservices/rest/search"
    params = {
        "query": query,
        "format": "json",
        "pageSize": 1  # 논문은 안 받고 개수만
    }

    response = requests.get(url, params=params)
    if response.status_code == 200:
        json_data = response.json()
        print("Raw 응답:", json_data)  # 디버깅용 출력
        return int(json_data.get("hitCount", 0))
    else:
        print(f"❌ API 요청 실패: {response.status_code}")
        return 0

query = '("antibiotic overuse" OR "vitamin C" OR "mRNA vaccine" OR "inflammation" OR "immunotherapy" OR "low carb diet" OR "mental health stigma" OR "supplements" OR "herbal medicine") AND PUB_YEAR:2025'
count = get_search_result_count(query)
print("🔎 검색된 논문 수:", count)


Raw 응답: {'version': '6.9', 'hitCount': 144937, 'nextCursorMark': 'AoIIQbTxkyg1MzA1Nzg5OQ==', 'nextPageUrl': 'https://www.ebi.ac.uk/europepmc/webservices/rest/search?query=("antibiotic overuse" OR "vitamin C" OR "mRNA vaccine" OR "inflammation" OR "immunotherapy" OR "low carb diet" OR "mental health stigma" OR "supplements" OR "herbal medicine") AND PUB_YEAR:2025&cursorMark=AoIIQbTxkyg1MzA1Nzg5OQ==&resultType=lite&pageSize=1&format=json', 'request': {'queryString': '("antibiotic overuse" OR "vitamin C" OR "mRNA vaccine" OR "inflammation" OR "immunotherapy" OR "low carb diet" OR "mental health stigma" OR "supplements" OR "herbal medicine") AND PUB_YEAR:2025', 'resultType': 'lite', 'cursorMark': '*', 'pageSize': 1, 'sort': '', 'synonym': False}, 'resultList': {'result': [{'id': '40433834', 'source': 'MED', 'pmid': '40433834', 'doi': '10.1111/pde.15998', 'title': 'Inflamed or Infected Molluscum Contagiosum Lesions: Pediatrician Perceptions and the Risk of Antibiotic Overuse.', 'authorStrin

In [2]:
import os
import time
import requests
from bs4 import BeautifulSoup
from pathlib import Path
from dotenv import load_dotenv
from concurrent.futures import ThreadPoolExecutor, as_completed
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain_openai import OpenAIEmbeddings
from langchain_chroma import Chroma
import tqdm
from dotenv import load_dotenv
import os

# ✅ 환경 변수 로딩 (.env에 OPENAI_API_KEY 포함되어야 함)
load_dotenv(dotenv_path="C:/Aicamp/SKN13_my/13_Langchain/.env")

# ✅ 글로벌 상수 설정
COLLECTION_NAME = "paper_db"
PERSIST_DIR = "vector_store/chroma"
EMBEDDING_MODEL = OpenAIEmbeddings(model="text-embedding-3-large")


def safe_request_get(url, params=None, headers=None, retries=5, backoff=2, timeout=10):
    """안정적인 GET 요청 함수 (재시도 포함)"""
    for attempt in range(retries):
        try:
            response = requests.get(url, params=params, headers=headers, timeout=timeout)
            response.raise_for_status()
            return response
        except requests.exceptions.RequestException as e:
            print(f"요청 실패 ({attempt + 1}/{retries}): {e}")
            time.sleep(backoff * (attempt + 1))
    raise Exception(f"최대 재시도 초과: {url}")


def extract_pmc_fulltext(pmcid):
    """PMC 논문 페이지에서 본문 텍스트 추출 (300자 이상)"""
    url = f"https://www.ncbi.nlm.nih.gov/pmc/articles/{pmcid}/"
    headers = {"User-Agent": "Mozilla/5.0"}
    res = safe_request_get(url, headers=headers)
    soup = BeautifulSoup(res.text, "html.parser")

    paragraphs = soup.select("p")
    text_blocks = [p.get_text(strip=True) for p in paragraphs if p.get_text(strip=True)]
    full_text = "\n\n".join(text_blocks).strip()

    if len(full_text) < 300:
        raise Exception(f"본문 길이 부족: {len(full_text)}자")

    return full_text, url


def chunk_text_to_documents(text, source_url, r):
    """본문을 Chunk로 분할하고 LangChain Document로 반환"""
    splitter = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=200)
    docs = splitter.create_documents([text])
    metadata = {
        "source": source_url,
        "pmcid": r.get("pmcid"),
        "title": r.get("title"),
        "doi": r.get("doi"),
        "pubYear": r.get("pubYear"),
        "journal": r.get("journalTitle"),
        "authors": r.get("authorString"),
        "abstract": r.get("abstractText")
    }
    for doc in docs:
        doc.metadata = metadata
    return docs


def get_existing_pmcids():
    """이미 Chroma DB에 저장된 PMC ID 목록 반환"""
    vector_store = Chroma(
        collection_name=COLLECTION_NAME,
        embedding_function=EMBEDDING_MODEL,
        persist_directory=PERSIST_DIR
    )
    metadatas = vector_store.get().get("metadatas", [])
    return {m.get("pmcid") for m in metadatas if m and m.get("pmcid")}


def store_documents(docs):
    """문서 리스트를 벡터 DB에 저장하고 결과 상태 반환"""
    pmcid = docs[0].metadata["pmcid"]
    existing_pmcs = get_existing_pmcids()

    if pmcid in existing_pmcs:
        return "중복으로 pass"

    try:
        vector_store = Chroma(
            collection_name=COLLECTION_NAME,
            embedding_function=EMBEDDING_MODEL,
            persist_directory=PERSIST_DIR
        )
        vector_store.add_documents(docs)
        return "성공"
    except Exception as e:
        return f"실패: {e}"


def process_single_article(r, current_index, total_count):
    """단일 논문 처리: 본문 추출 → chunk → 저장 → 결과 메시지 반환"""
    pmcid = r.get("pmcid")
    if not pmcid:
        return f"[{current_index}/{total_count}] PMCID 없음 → 실패"

    try:
        text, url = extract_pmc_fulltext(pmcid)
        docs = chunk_text_to_documents(text, url, r)
        result = store_documents(docs)
        return f"[{current_index}/{total_count}] {pmcid} 처리 결과: {result}"
    except Exception as e:
        return f"[{current_index}/{total_count}] {pmcid} 처리 실패: {str(e)}"


def process_all_pmc_articles(query):
    """
    유럽 PMC API를 통해 논문 메타데이터를 수집하고,
    각 논문을 병렬로 처리하여 벡터 DB에 저장합니다.
    """
    cursor = "*"
    page_size = 25
    index_counter = 1
    total_expected = None
    existing_pmcs = get_existing_pmcids()

    while True:
        res = safe_request_get(
            "https://www.ebi.ac.uk/europepmc/webservices/rest/search",
            params={"query": query, "format": "json", "pageSize": page_size, "cursorMark": cursor}
        )
        data = res.json()
        results = data.get("resultList", {}).get("result", [])

        if not results:
            break

        filtered = [r for r in results if r.get("pmcid") not in existing_pmcs]
        total_expected = total_expected or data.get("hitCount", len(filtered))

        with ThreadPoolExecutor(max_workers=5) as executor:
            futures = [
                executor.submit(process_single_article, r, index_counter + i, total_expected)
                for i, r in enumerate(filtered)
            ]
            for future in as_completed(futures):
                print(future.result())

        cursor = data.get("nextCursorMark")
        index_counter += len(results)
        if not cursor:
            break


In [3]:
if __name__ == "__main__":
    query = '("coffee consumption" AND "health") AND PUB_YEAR:2025'
    process_all_pmc_articles(query)

[4/400] PMCID 없음 → 실패
[6/400] PMCID 없음 → 실패
[5/400] PMCID 없음 → 실패
[9/400] PMCID 없음 → 실패
[1/400] PMCID 없음 → 실패
[7/400] PMCID 없음 → 실패
[10/400] PMCID 없음 → 실패
[12/400] PMCID 없음 → 실패
[13/400] PMC11944175 처리 결과: 성공
[14/400] PMCID 없음 → 실패
[15/400] PMCID 없음 → 실패
[11/400] PMC12021940 처리 결과: 성공
[17/400] PMCID 없음 → 실패
[18/400] PMCID 없음 → 실패
[19/400] PMCID 없음 → 실패
[2/400] PMC11909266 처리 결과: 성공
[21/400] PMCID 없음 → 실패
[8/400] PMC12037506 처리 결과: 성공
[23/400] PMCID 없음 → 실패
[24/400] PMCID 없음 → 실패
[25/400] PMCID 없음 → 실패
[3/400] PMC12073867 처리 결과: 성공
요청 실패 (1/5): HTTPSConnectionPool(host='www.ncbi.nlm.nih.gov', port=443): Max retries exceeded with url: /pmc/articles/PMC11978573/ (Caused by ConnectTimeoutError(<urllib3.connection.HTTPSConnection object at 0x000001C1EA7DA600>, 'Connection to www.ncbi.nlm.nih.gov timed out. (connect timeout=10)'))
[20/400] PMC11853883 처리 결과: 성공
[16/400] PMC11948776 처리 결과: 성공
[22/400] PMC11978573 처리 결과: 성공
[27/400] PMCID 없음 → 실패
[26/400] PMCID 없음 → 실패
[28/400] PMCID 없음 → 실패
[

In [ ]:
# # 유럽 PMC 논문 크롤링 + 전처리 + 벡터 DB 저장 (RAG용 최적화 리팩터링)

# import os
# import time
# import requests
# from bs4 import BeautifulSoup
# from pathlib import Path
# from dotenv import load_dotenv
# from concurrent.futures import ThreadPoolExecutor, as_completed
# from langchain.text_splitter import RecursiveCharacterTextSplitter
# from langchain_core.documents import Document
# from langchain_openai import OpenAIEmbeddings
# from langchain_chroma import Chroma
# import tqdm

# # .env 파일에서 환경변수 불러오기
# load_dotenv(dotenv_path=Path("C:/Aicamp/SKN13_my/13_Langchain/.env"))

# def safe_request_get(url, params=None, headers=None, retries=5, backoff=2, timeout=10):
#     """
#     안전한 GET 요청 함수. 실패 시 재시도(backoff 포함).
#     """
#     for attempt in range(retries):
#         try:
#             print(f"요청 중: {url}")
#             response = requests.get(url, params=params, headers=headers, timeout=timeout)
#             response.raise_for_status()
#             return response
#         except requests.exceptions.RequestException as e:
#             print(f"요청 실패 ({attempt+1}/{retries}): {e}")
#             time.sleep(backoff * (attempt + 1))
#     raise Exception(f"요청 실패: {url}")

# def extract_pmc_fulltext(pmcid):
#     """
#     PMC 논문의 본문 HTML에서 <p> 태그만 추출하여 텍스트로 반환합니다.
#     300자 미만일 경우 예외를 발생시킵니다.
#     """
#     url = f"https://www.ncbi.nlm.nih.gov/pmc/articles/{pmcid}/"
#     headers = {"User-Agent": "Mozilla/5.0"}
#     res = safe_request_get(url, headers=headers)
#     soup = BeautifulSoup(res.text, "html.parser")

#     sections = soup.select("p")
#     if not sections:
#         raise Exception(f"본문 p 태그 없음: {url}")

#     text_blocks = [p.get_text(strip=True) for p in sections if p.get_text(strip=True)]
#     if not text_blocks:
#         raise Exception(f"텍스트 블록 없음: {url}")

#     full_text = "\n\n".join(text_blocks).strip()
#     if len(full_text) < 300: # 내용이 짧아, 신뢰성 결여
#         raise Exception(f"내용 너무 짧음 ({len(full_text)}자): {url}")

#     return full_text, url

# def chunk_text_to_documents(text, source_url, r):
#     """
#     본문 텍스트를 일정 길이로 분할하고, 논문 메타데이터를 부여하여 Document 리스트를 생성합니다.
#     """
#     splitter = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=200) # 업계 표준 설정
#     docs = splitter.create_documents([text])
#     metadata_common = {
#         "source": source_url,
#         "pmcid": r.get("pmcid"),
#         "title": r.get("title"),
#         "doi": r.get("doi"),
#         "pubYear": r.get("pubYear"),
#         "journal": r.get("journalTitle"),
#         "authors": r.get("authorString"),
#         "abstract": r.get("abstractText")
#     }
#     for doc in docs:
#         doc.metadata = metadata_common
#     return docs

# def store_documents_to_chroma(docs, collection_name="paper_db", persist_dir="vector_store/chroma", doc_counter=[0]):
#     """
#     Chroma 벡터 DB에 문서 리스트를 저장합니다. 중복 PMC ID는 건너뜁니다.
#     """
#     embedding_model = OpenAIEmbeddings(model="text-embedding-3-large")
#     vector_store = Chroma(
#         collection_name=collection_name,
#         embedding_function=embedding_model,
#         persist_directory=persist_dir
#     )
#     existing_pmcs = {
#         metadata.get("pmcid") for metadata in vector_store.get().get("metadatas", []) if metadata
#     }
#     new_docs = [doc for doc in docs if doc.metadata["pmcid"] not in existing_pmcs]

#     if new_docs:
#         vector_store.add_documents(new_docs)
#         doc_counter[0] += 1
#         print(f"문서 저장 완료 (총 {doc_counter[0]}번째 논문)")
#     else:
#         print("중복 문서로 저장 생략")

# def count_documents(collection_name="paper_db", persist_dir="vector_store/chroma"):
#     """
#     Chroma DB에 저장된 전체 문서 수를 반환합니다.
#     """
#     vector_store = Chroma(
#         collection_name=collection_name,
#         persist_directory=persist_dir,
#         embedding_function=OpenAIEmbeddings(model="text-embedding-3-large")
#     )
#     return len(vector_store.get().get("documents", []))

# def count_existing_articles(collection_name="paper_db", persist_dir="vector_store/chroma"):
#     """
#     중복 방지를 위해 Chroma DB에 저장된 고유 PMC ID 수를 반환합니다.
#     """
#     vector_store = Chroma(
#         collection_name=collection_name,
#         persist_directory=persist_dir,
#         embedding_function=OpenAIEmbeddings(model="text-embedding-3-large")
#     )
#     metadatas = vector_store.get().get("metadatas", [])
#     pmcids = {m.get("pmcid") for m in metadatas if m and m.get("pmcid")}
#     return len(pmcids)

# def get_existing_pmcids(collection_name="paper_db", persist_dir="vector_store/chroma"):
#     """
#     벡터 DB에서 현재 저장된 PMC ID 목록을 반환합니다.
#     """
#     vector_store = Chroma(
#         collection_name=collection_name,
#         persist_directory=persist_dir,
#         embedding_function=OpenAIEmbeddings(model="text-embedding-3-large")
#     )
#     metadatas = vector_store.get().get("metadatas", [])
#     return {m.get("pmcid") for m in metadatas if m and m.get("pmcid")}

# def process_single_article(r, index, doc_counter):
#     """
#     단일 논문 처리 루틴.
#     - 본문 추출
#     - 분할 및 메타데이터 추가
#     - 벡터 DB 저장
#     """
#     pmcid = r.get("pmcid")
#     if not pmcid:
#         print("PMCID 없음")
#         return False
#     print(f"논문 {index}: {pmcid} - {r.get('title')}")
#     try:
#         text, source_url = extract_pmc_fulltext(pmcid)
#         docs = chunk_text_to_documents(text, source_url, r)
#         store_documents_to_chroma(docs, doc_counter=doc_counter)
#         return True
#     except Exception as e:
#         print(f"처리 실패: {e}")
#         return False



# def process_all_pmc_articles(query):
#     """
#     Europe PMC API를 통해 질의(query)에 해당하는 논문들을 수집하고 처리합니다.
#     - 중복 PMCID는 건너뜁니다
#     - 병렬 처리 (ThreadPoolExecutor)
#     - 크롤링 재개 가능성 고려 (중단 시 이어서 실행)
#     """
#     cursor = "*"
#     page_size = 25
#     total = 0
#     index_counter = 1
#     doc_counter = [count_existing_articles()]
#     existing_pmcids = get_existing_pmcids()

#     pbar = tqdm.tqdm(desc="전체 처리", unit="논문")

#     while True:
#         res = safe_request_get(
#             "https://www.ebi.ac.uk/europepmc/webservices/rest/search",
#             params={"query": query, "format": "json", "pageSize": page_size, "cursorMark": cursor}
#         )
#         data = res.json()
#         results = data.get("resultList", {}).get("result", [])
#         if not results:
#             break

#         filtered_results = [r for r in results if r.get("pmcid") not in existing_pmcids]

#         with ThreadPoolExecutor(max_workers=5) as executor:
#             futures = {
#                 executor.submit(process_single_article, r, index, doc_counter): r
#                 for index, r in enumerate(filtered_results, start=index_counter)
#             }
#             for f in as_completed(futures):
#                 if f.result():
#                     total += 1
#                     pbar.update(1)

#         index_counter += len(results)
#         cursor = data.get("nextCursorMark")
#         if not cursor:
#             break

#     pbar.close()
#     print(f"저장한 논문 수: {total}")
#     print(f"현재 DB 내 전체 문서 수: {count_documents()}개")

In [ ]:
# if __name__ == "__main__":
#     query = '("antibiotic overuse" OR "vitamin C" OR "mRNA vaccine" OR "inflammation" OR "immunotherapy" OR "low carb diet" OR "mental health stigma" OR "supplements" OR "herbal medicine") AND PUB_YEAR:2025'
#     process_all_pmc_articles(query)